In [1]:
sentence = 'Life is short, eat dessert first'

dc = {s:i for i,s in enumerate(sorted(sentence.replace(',', '').split()))}
print(dc)

{'Life': 0, 'dessert': 1, 'eat': 2, 'first': 3, 'is': 4, 'short': 5}


In [2]:
import torch

sentence_int = torch.tensor([dc[s] for s in sentence.replace(',', '').split()])
print(sentence_int)

tensor([0, 4, 5, 2, 1, 3])


In [3]:
torch.manual_seed(123)
embed = torch.nn.Embedding(6, 16)
embedded_sentence = embed(sentence_int).detach()

print(embedded_sentence)
print(embedded_sentence.shape)

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
          0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692],
        [ 0.5146,  0.9938, -0.2587, -1.0826, -0.0444,  1.6236, -2.3229,  1.0878,
          0.6716,  0.6933, -0.9487, -0.0765, -0.1526,  0.1167,  0.4403, -1.4465],
        [ 0.2553, -0.5496,  1.0042,  0.8272, -0.3948,  0.4892, -0.2168, -1.7472,
         -1.6025, -1.0764,  0.9031, -0.7218, -0.5951, -0.7112,  0.6230, -1.3729],
        [-1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343, -0.4991, -1.0867,
          0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [-0.0770, -1.0205, -0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010,
          0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255, -0.6315, -2.8400],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
          2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,  0.2293]])
torch.Size([6, 16])


In [13]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]

d_q, d_k, d_v = 8,8,16

W_query = torch.nn.Parameter(torch.rand(d_q, d))
W_key = torch.nn.Parameter(torch.rand(d_k, d))
W_value = torch.nn.Parameter(torch.rand(d_v, d))

In [14]:
keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T
queries = W_query.matmul(embedded_sentence.T).T
print("queries.shape:", queries.shape)
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

queries.shape: torch.Size([6, 8])
keys.shape: torch.Size([6, 8])
values.shape: torch.Size([6, 16])


In [15]:
import torch.nn.functional as F

In [16]:
# getting query * keys^T
qt = queries.matmul(keys.T)
print("qt.shape:", qt.shape)
print("qt:", qt)

qt.shape: torch.Size([6, 6])
qt: tensor([[-9.0400e-01, -3.0810e-02,  2.7150e+00, -3.6349e+00,  5.6220e+00,
         -1.3809e+00],
        [-1.8299e+00, -9.1797e-01, -5.3252e-01,  1.4209e-01, -9.0604e-01,
          3.7870e+00],
        [-8.5605e+00,  2.9286e+00,  5.2545e+01,  9.3362e+00,  1.6665e+01,
         -5.1919e+01],
        [-7.6334e+00, -1.9176e-02,  3.6146e+01,  1.0520e+01,  4.9118e+00,
         -3.1107e+01],
        [-1.0616e+01, -1.7377e+00,  4.8166e+01,  5.9618e+00,  1.3659e+01,
         -4.6702e+01],
        [ 1.1098e+01, -3.0143e+00, -5.2936e+01, -8.9371e+00, -1.8481e+01,
          5.1137e+01]], grad_fn=<MmBackward0>)


In [17]:

# scaling
qt = qt / (d_k ** 0.5)
print("scaled qt.shape:", qt.shape)
print("scaled qt:", qt)

scaled qt.shape: torch.Size([6, 6])
scaled qt: tensor([[-3.1961e-01, -1.0893e-02,  9.5989e-01, -1.2851e+00,  1.9877e+00,
         -4.8823e-01],
        [-6.4698e-01, -3.2455e-01, -1.8827e-01,  5.0236e-02, -3.2033e-01,
          1.3389e+00],
        [-3.0266e+00,  1.0354e+00,  1.8577e+01,  3.3009e+00,  5.8919e+00,
         -1.8356e+01],
        [-2.6988e+00, -6.7798e-03,  1.2779e+01,  3.7196e+00,  1.7366e+00,
         -1.0998e+01],
        [-3.7533e+00, -6.1437e-01,  1.7029e+01,  2.1078e+00,  4.8291e+00,
         -1.6511e+01],
        [ 3.9236e+00, -1.0657e+00, -1.8716e+01, -3.1597e+00, -6.5341e+00,
          1.8080e+01]], grad_fn=<DivBackward0>)


In [18]:
# softmax
qt = F.softmax(qt, dim=-1)
print("softmax qt.shape:", qt.shape)
print("softmax qt:", qt)

softmax qt.shape: torch.Size([6, 6])
softmax qt: tensor([[5.8040e-02, 7.9032e-02, 2.0865e-01, 2.2101e-02, 5.8315e-01, 4.9034e-02],
        [6.8294e-02, 9.4278e-02, 1.0804e-01, 1.3714e-01, 9.4677e-02, 4.9756e-01],
        [4.1452e-10, 2.4079e-08, 1.0000e+00, 2.3202e-07, 3.0959e-06, 9.1212e-17],
        [1.8960e-07, 2.7988e-06, 9.9986e-01, 1.1623e-04, 1.6000e-05, 4.7154e-11],
        [9.4244e-10, 2.1750e-08, 9.9999e-01, 3.3089e-07, 5.0297e-06, 2.7129e-15],
        [7.1142e-07, 4.8449e-09, 1.0470e-16, 5.9684e-10, 2.0436e-11, 1.0000e+00]],
       grad_fn=<SoftmaxBackward0>)


In [19]:
# getting attention
att = qt.matmul(values)
print("att.shape:", att.shape)
print("att:", att)

att.shape: torch.Size([6, 16])
att: tensor([[-1.9141,  0.0725, -1.1123, -1.7328, -0.3719, -1.3051, -1.7193, -1.1762,
          0.3699,  0.3711,  0.1109, -0.0922, -2.1684, -0.4655, -1.0411, -1.2061],
        [ 0.1633,  0.9595, -0.6752,  0.4028, -0.7745, -0.5496,  1.4194, -0.3004,
          0.9839,  2.0319,  0.4223,  1.4705, -0.0272,  1.1352,  1.0462,  0.3647],
        [-3.2055, -1.5096, -2.4377, -2.7335, -1.7701, -0.5160, -3.7293, -2.7705,
         -2.1744, -2.7339, -1.0410, -1.8867, -4.0902, -0.3303, -3.1343, -2.4864],
        [-3.2053, -1.5095, -2.4377, -2.7333, -1.7700, -0.5161, -3.7290, -2.7704,
         -2.1741, -2.7335, -1.0411, -1.8864, -4.0899, -0.3307, -3.1338, -2.4863],
        [-3.2055, -1.5096, -2.4377, -2.7335, -1.7701, -0.5160, -3.7293, -2.7705,
         -2.1744, -2.7339, -1.0410, -1.8867, -4.0902, -0.3303, -3.1343, -2.4864],
        [ 1.8757,  1.9398, -0.1509,  1.9660, -0.5092, -0.2307,  4.1125,  0.5593,
          2.1795,  4.3551,  1.7887,  3.0898,  1.5155,  3.2049,  2.53